In [1]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
## Librerias necesarias
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib
import plotly
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


In [63]:
###Extraer la información de yahoo finance
acciones=["^GSPC","AAPL","AMZN","MSFT","TSLA","GOOG","GOOGL","NVDA","BRK-B","META","UNH","JNJ","PG"]
acciones = yf.download(acciones, start="2012-05-21", end="2023-03-01")["Adj Close"]
acciones.tail()

[*********************100%***********************]  13 of 13 completed


,AAPL,AMZN,BRK-B,GOOG,GOOGL,JNJ,META,MSFT,NVDA,PG,TSLA,UNH,^GSPC
Date,,,,,,,,,,,,,
2023-02-22,148.910004,95.790001,303.160004,91.800003,91.650002,157.779999,171.119995,251.509995,207.504745,139.625824,200.860001,487.154907,3991.050049
2023-02-23,149.399994,95.820000,303.070007,91.070000,90.889999,157.729996,172.039993,254.770004,236.599808,139.178635,202.070007,489.944946,4012.320068
2023-02-24,146.710007,93.500000,304.019989,89.349998,89.129997,155.970001,170.389999,249.220001,232.820465,138.393539,196.880005,482.611053,3970.040039
2023-02-27,147.919998,93.760002,304.660004,90.099998,89.870003,155.630005,169.539993,250.160004,234.970093,138.274292,207.630005,481.604645,3982.239990
2023-02-28,147.410004,94.230003,305.179993,90.300003,90.059998,153.259995,174.940002,249.419998,232.120575,136.704117,205.710007,474.250854,3970.149902


In [66]:
###Calcular los retornos logaritmicos
log_returns=(np.log(1+acciones.pct_change()))*100
log_returns.tail()

##Se elimina la primera fila
log_returns=log_returns.dropna()

In [67]:
log_returns.iloc[:,12]

Date
2012-05-22    0.048622
2012-05-23    0.169227
2012-05-24    0.137908
2012-05-25   -0.216798
2012-05-29    1.101806
                ...   
2023-02-22   -0.157480
2023-02-23    0.531528
2023-02-24   -1.059346
2023-02-27    0.306829
2023-02-28   -0.304062
Name: ^GSPC, Length: 2710, dtype: float64

In [71]:
X=log_returns.iloc[:,12]
X= sm.add_constant(X, prepend=True)
modelo = sm.OLS(endog=log_returns.iloc[:,1], exog=X)
modelo = modelo.fit()
print(modelo.summary())

###Valores de alpha, valor p alpha , beta, valor p beta



                            OLS Regression Results                            
Dep. Variable:                   AMZN   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     1613.
Date:                Sat, 06 May 2023   Prob (F-statistic):          4.76e-277
Time:                        01:34:28   Log-Likelihood:                -5145.5
No. Observations:                2710   AIC:                         1.029e+04
Df Residuals:                    2708   BIC:                         1.031e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0333      0.031      1.071      0.2

In [74]:
##Alpha
modelo.params[0]
##Beta
modelo.params[1]

0.03328537307299881

In [76]:
###Valor p
modelo.pvalues[0]

0.2841191582106192

In [77]:
###Valor p
modelo.pvalues[1]

4.757302564136318e-277

In [87]:
Accion=[]
Alpha=[]
ValorPAlpha=[]
Beta=[]
ValorPBeta=[]

for i in range(0,(len(log_returns.columns)-1)):
    X=log_returns.iloc[:,12]
    X= sm.add_constant(X, prepend=True)
    modelo = sm.OLS(endog=log_returns.iloc[:,i], exog=X)
    modelo = modelo.fit()
    
    Accion.append(log_returns.columns[i])
    Alpha.append(round(modelo.params[0],3))
    Beta.append(round(modelo.params[1],3))
    ValorPAlpha.append(round(modelo.pvalues[0],3))
    ValorPBeta.append(round(modelo.pvalues[1],3))  

result = list(zip(Accion,Alpha, ValorPAlpha, Beta,ValorPBeta))
df = pd.DataFrame(result)
df = df.rename(columns={0:'Accion',1:'Alpha',2:'Valor-p Alpha',3:'Beta',4:'Valor-p Beta' })
df

,Accion,Alpha,Valor-p Alpha,Beta,Valor-p Beta
0,AAPL,0.032,0.202,1.170,0.0
1,AMZN,0.033,0.284,1.136,0.0
2,BRK-B,0.013,0.315,0.887,0.0
3,GOOG,0.020,0.382,1.123,0.0
4,GOOGL,0.019,0.389,1.127,0.0
5,JNJ,0.019,0.247,0.598,0.0
6,META,0.011,0.799,1.220,0.0
7,MSFT,0.037,0.067,1.196,0.0
8,NVDA,0.097,0.017,1.622,0.0
9,PG,0.016,0.366,0.589,0.0
